In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Basic Notebook for Train in Inference

In [ ]:
BATCH_SIZE = 16


In [ ]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification,RobertaForSequenceClassification
from sklearn.model_selection import train_test_split

# Same as before
checkpoint = "../input/clrp-roberta-base/clrp_roberta_base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint,from_pt=True)
base_model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint,from_pt=True)

VERBOSE = 1



df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

seqs = df.excerpt
targets = df.target

df_test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

test_seqs = df_test.excerpt


# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(seqs, targets, test_size=0.2, shuffle= True)




In [ ]:
x_train[:1]

In [ ]:
batch = dict(tokenizer(seqs.to_list(), padding=True, truncation=True, return_tensors="tf"))

test_batch = dict(tokenizer(test_seqs.to_list(), padding=True, truncation=True, return_tensors="tf"))

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(f'bert_Base.h5', 
                                                        monitor = 'val_root_mean_squared_error', 
                                                        verbose = VERBOSE, 
                                                        save_best_only = True,
                                                        save_weights_only = True, 
                                                        mode = 'min') 
            ]

In [ ]:
steps = x_train.shape[0] // (BATCH_SIZE *16)

In [ ]:
base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5,epsilon=1e-6),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics = [tf.keras.metrics.RootMeanSquaredError()],
             )

labels = tf.convert_to_tensor(targets)

history = base_model.fit(x=batch,y=labels,batch_size=BATCH_SIZE,epochs=70
                   ,callbacks = [callbacks],
                         steps_per_epoch = steps,
                         validation_split=0.2
                   )

In [ ]:
base_model.summary()

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['root_mean_squared_error', 'val_root_mean_squared_error']].plot();


# Predict

In [ ]:
base_model.load_weights(f'./bert_Base.h5')
    
preds = base_model.predict(test_batch)

In [ ]:
df_test['target'] = np.mean(preds[0],axis=1)
df_test[['id', 'target']].to_csv('submission.csv', index = False)